In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from flask import Flask, jsonify,render_template
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from mysql_conn import password
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
import requests
import pandas as pd

In [2]:
connection_string = (f"root:{password}@localhost/real_estate")
engine = create_engine(f"mysql://{connection_string}")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save reference to the table
mean_sales_count = Base.classes.mean_sales_count
median_sales_count = Base.classes.median_sales_count
median_price_sqft = Base.classes.median_price_sqft
median_price_zip = Base.classes.median_price_zip
print(Base.classes.keys())

# Create our connection object
session = Session(engine)

['mean_sales_count', 'median_price_sqft', 'median_price_zip', 'median_sales_count']


In [3]:
city_dict = {}
results = (session.query(mean_sales_count.Metro).all())
city_list = list(np.ravel(results))
print(city_list)

city_ST = {'Austin-Round Rock': 'Austin, TX', 
            'Dallas-Fort Worth-Arlington': 'Dallas-Forth Worth, TX', 
            'Denver-Aurora-Lakewood': 'Denver, CO', 
            'Detroit-Warren-Dearborn': 'Detroit, MI', 
            'New York-Newark-Jersey City': 'New York City, NY', 
            'Orlando-Kissimmee-Sanford': 'Orlando, FL', 
            'Raleigh-Durham-Chapel Hill': 'Raleigh-Durham, NC', 
            'San Francisco-Oakland-Hayward': 'San Francisco, CA', 
            'Seattle-Tacoma-Bellevue': 'Seattle, WA', 
            'Washington-Arlington-Alexandria': 'Washington D.C.'}

['Austin-Round Rock', 'Dallas-Fort Worth-Arlington', 'Denver-Aurora-Lakewood', 'Detroit-Warren-Dearborn', 'New York-Newark-Jersey City', 'Orlando-Kissimmee-Sanford', 'Raleigh-Durham-Chapel Hill', 'San Francisco-Oakland-Hayward', 'Seattle-Tacoma-Bellevue', 'Washington-Arlington-Alexandria']


In [10]:
city_dict = {}
for city in city_list[2:]:
    price_results = ((session.query(median_price_zip._2008,
                                        median_price_zip._2009,
                                        median_price_zip._2010,
                                        median_price_zip._2011,
                                        median_price_zip._2012,
                                        median_price_zip._2013,
                                        median_price_zip._2014,
                                        median_price_zip._2015,
                                        median_price_zip._2016,
                                        median_price_zip._2017,
                                        median_price_zip._2018)).filter(median_price_zip.Metro == city).all())

    price_results_list = list(np.ravel(price_results))
    
    city_dict[city_ST[city]] = {
        "Metro Area": city,
        "Median Sale Price Per Year": {
            "2008": str(price_results_list[0]),
            "2009": str(price_results_list[1]),
            "2010": str(price_results_list[2]),
            "2011": str(price_results_list[3]),
            "2012": str(price_results_list[4]),
            "2013": str(price_results_list[5]),
            "2014": str(price_results_list[6]),
            "2015": str(price_results_list[7]),
            "2016": str(price_results_list[8]),
            "2017": str(price_results_list[9]),
            "2018": str(price_results_list[10])
        }}
    print(city_dict)

{'Denver, CO': {'Metro Area': 'Denver-Aurora-Lakewood', 'Median Sale Price Per Year': {'2008': '2124100.0000000000', '2009': '2480350.0000000000', '2010': '2551150.0000000000', '2011': '2537500.0000000000', '2012': '2700000.0000000000', '2013': '2924150.0000000000', '2014': '3201450.0000000000', '2015': '3594500.0000000000', '2016': '4057150.0000000000', '2017': '4423900.0000000000', '2018': '4840450.0000000000'}}}
{'Denver, CO': {'Metro Area': 'Denver-Aurora-Lakewood', 'Median Sale Price Per Year': {'2008': '2124100.0000000000', '2009': '2480350.0000000000', '2010': '2551150.0000000000', '2011': '2537500.0000000000', '2012': '2700000.0000000000', '2013': '2924150.0000000000', '2014': '3201450.0000000000', '2015': '3594500.0000000000', '2016': '4057150.0000000000', '2017': '4423900.0000000000', '2018': '4840450.0000000000'}}, 'Detroit, MI': {'Metro Area': 'Detroit-Warren-Dearborn', 'Median Sale Price Per Year': {'2008': '1438250.0000000000', '2009': '1393100.0000000000', '2010': '12650